In [2]:
from uncertainty_analysis import random_forest_ensemble as rfe
import pandas as pd
import openml

In [3]:
dataset = openml.datasets.get_dataset(44061)
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

C:\Users\elsag\OneDrive\Desktop\Uni\Year 4\Academic\Dissertation\Final Report\Code\scikit-learn-1.2.1\scikit-learn-1.2.1\venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [4]:
columns_to_encode = ['X3', 'X4', 'X6']
X = pd.get_dummies(X, columns=columns_to_encode)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
merc_rf_ens = []
merc_rf_ens_preds = [] # List[ensemble np.array[regressors np.array[predictions]]
for i in range(0, 10):
    m = rfe.RandomForestRegressorEnsemble(n_jobs=5, n_regressors=100, criterion='poisson',
                                                 max_depth=None, max_features=50, min_samples_leaf=5,
                                                 min_samples_split=30, n_estimators=600).fit(X_train, y_train)
    merc_rf_ens.append(m)
    merc_rf_ens_preds.append(m.predict(X_test.apply(pd.to_numeric)))

In [7]:
from uncertainty_analysis import project_helper_functions as phf

best_model_results = merc_rf_ens_preds[phf.get_best_model_idx(merc_rf_ens_preds, y_test)]
phf.write_ensemble_model_results(best_model_results.T, "merc_ensemble_rf_results.txt")